<a href="https://colab.research.google.com/github/ajijohn/sathack/blob/master/planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import requests
from requests.auth import HTTPBasicAuth
import json

os.environ['PL_API_KEY'] = '332d82b813b64703a6222bb42e32f27c'

geo_json_geometry = {
  "type": "Polygon",
  "coordinates": [
    [
      [
        -122.52227783203125,
        40.660847697284815
      ],
      [
        -122.52227783203125,
        40.987154933797335
      ],
      [
        -122.01690673828124,
        40.987154933797335
      ],
      [
        -122.01690673828124,
        40.660847697284815
      ],
      [
        -122.52227783203125,
        40.660847697284815
      ]
    ]
  ]
}

# filter for items the overlap with our chosen geometry
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geo_json_geometry
}

# filter images acquired in a certain date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2016-07-01T00:00:00.000Z",
    "lte": "2016-08-01T00:00:00.000Z"
  }
}

# filter any images which are more than 50% clouds
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# create a filter that combines our geo and date filters
# could also use an "OrFilter"
redding_reservoir = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [10]:


# Stats API request object
stats_endpoint_request = {
  "interval": "day",
  "item_types": ["REOrthoTile"],
  "filter": redding_reservoir
}

# fire off the POST request
result = \
  requests.post(
    'https://api.planet.com/data/v1/stats',
    auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''),
    json=stats_endpoint_request)

print(result.text)

{"utc_offset": "+0h", "interval": "day", "buckets": [{"count": 6, "start_time": "2016-07-07T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-08T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-09T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-10T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-11T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-12T00:00:00.000000Z"}, {"count": 9, "start_time": "2016-07-13T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-14T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-15T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-16T00:00:00.000000Z"}, {"count": 3, "start_time": "2016-07-17T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-18T00:00:00.000000Z"}, {"count": 6, "start_time": "2016-07-19T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-20T00:00:00.000000Z"}, {"count": 0, "start_time": "2016-07-21T00:00:00.000000Z"}, {"count": 6, "start_time": "2016-07-22T00:00:00.000000Z"}]}


In [11]:
# Search API request object
search_endpoint_request = {
  "item_types": ["REOrthoTile"],
  "filter": redding_reservoir
}

result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(os.environ['PL_API_KEY'], ''),
    json=search_endpoint_request)

print(result.text)

{"_links": {"_first": "https://api.planet.com/data/v1/searches/1575c7fd247f4251b11812c5f0e71a3a/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D", "_next": "https://api.planet.com/data/v1/searches/1575c7fd247f4251b11812c5f0e71a3a/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6ICIyMDE2LTA4LTE4VDAxOjUzOjQwLjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMTYwNzE5XzE5Mzg0MV8xMDU3ODE3X1JhcGlkRXllLTQiLCAic29ydF9kZXNjIjogdHJ1ZX0%3D", "_self": "https://api.planet.com/data/v1/searches/1575c7fd247f4251b11812c5f0e71a3a/results?_page=eyJxdWVyeV9wYXJhbXMiOiB7fSwgInNvcnRfcHJldiI6IGZhbHNlLCAicGFnZV9zaXplIjogMjUwLCAic29ydF9ieSI6ICJwdWJsaXNoZWQiLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9kZXNjIjogdHJ1ZX0%3D"}, "features": [{"_links": {"_s

In [0]:
rm=json.loads(result.text)
rm['features']
imageids=[]
for feature in rm['features']:
  imageids.append(feature['id'])

In [21]:
print(imageids)


['20160707_195147_1057916_RapidEye-1', '20160707_195146_1057917_RapidEye-1', '20160707_195150_1057817_RapidEye-1', '20160707_195143_1058017_RapidEye-1', '20160707_195143_1058016_RapidEye-1', '20160707_195150_1057816_RapidEye-1', '20160722_194931_1057916_RapidEye-2', '20160722_194930_1057917_RapidEye-2', '20160722_194928_1058016_RapidEye-2', '20160722_194927_1058017_RapidEye-2', '20160722_194935_1057816_RapidEye-2', '20160722_194934_1057817_RapidEye-2', '20160717_194449_1057918_RapidEye-2', '20160717_194445_1058018_RapidEye-2', '20160717_194452_1057818_RapidEye-2', '20160713_193341_1057918_RapidEye-3', '20160713_193341_1057917_RapidEye-3', '20160713_193342_1057916_RapidEye-3', '20160713_193339_1058016_RapidEye-3', '20160713_193338_1058017_RapidEye-3', '20160713_193337_1058018_RapidEye-3', '20160713_193345_1057817_RapidEye-3', '20160713_193345_1057816_RapidEye-3', '20160713_193344_1057818_RapidEye-3', '20160719_193838_1057917_RapidEye-4', '20160719_193838_1057916_RapidEye-4', '20160719_1

In [0]:
image1='20160707_195147_1057916_RapidEye-1'

In [25]:
import os
import requests

item_id = image1
item_type = "REOrthoTile"
asset_type = "visual"

# setup auth
session = requests.Session()
session.auth = (os.environ['PL_API_KEY'], '')

# request an item
item = \
  session.get(
    ("https://api.planet.com/data/v1/item-types/" +
    "{}/items/{}/assets/").format(item_type, item_id))

# extract the activation url from the item for the desired asset
item_activation_url = item.json()[asset_type]["_links"]["activate"]

# request activation
response = session.post(item_activation_url)

print (response.status_code)


204


In [27]:
item.json()[asset_type]["status"]

'active'

In [0]:
payload = {
   "aoi" : geo_json_geometry,
   "targets" : [{
       "item_id" : item_id,
       "item_type" : item_type,
       "asset_type" : asset_type
   }]
}

request = requests.post('https://api.planet.com/compute/ops/clips/v1', auth=(os.environ['PL_API_KEY'], ''), json=payload)
clip_url = request.json()['_links']['_self']

In [0]:
check_state_request = requests.get(clip_url, auth=(os.environ['PL_API_KEY'], ''))

In [38]:
check_state_request.json()

{'_links': {'_self': 'https://api.planet.com/compute/ops/clips/v1/760f664c-e629-4252-b0ab-eaf79200d955',
  'results': ['https://api.planet.com/data/v1/download?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1NTY4NzU0ODMsInN1YiI6InFsdlNuUFpMRGxQTGl6aDZGZUFrOVR2cUN2dXErZGN4SEhhQUlWR3IzbDdWS1huSDR1NmNPRzdPK256aytPSGxwejdjZ1hzUnUxVFBHcnZhWnFDOCtnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiJ7XCJ0eXBlXCI6IFwiUG9seWdvblwiLCBcImNvb3JkaW5hdGVzXCI6IFtbWy0xMjIuNTIyMjc3ODMyMDMxMjUsIDQwLjY2MDg0NzY5NzI4NDgxNV0sIFstMTIyLjUyMjI3NzgzMjAzMTI1LCA0MC45ODcxNTQ5MzM3OTczMzVdLCBbLTEyMi4wMTY5MDY3MzgyODEyNCwgNDAuOTg3MTU0OTMzNzk3MzM1XSwgWy0xMjIuMDE2OTA2NzM4MjgxMjQsIDQwLjY2MDg0NzY5NzI4NDgxNV0sIFstMTIyLjUyMjI3NzgzMjAzMTI1LCA0MC42NjA4NDc2OTcyODQ4MTVdXV19IiwiYXNzZXRzIjpbeyJpdGVtX3R5cGUiOiJSRU9ydGhvVGlsZSIsImFzc2V0X3R5cGUiOiJ2aXN1YWwiLCJpdGVtX2lkIjoiMjAxNjA3MDdfMTk1MTQ3XzEwNTc5MTZfUmFwaWRFeWUtMSJ9XSwidXJsIjoiaHR0cHM6Ly9wbGFuZXQtcGxhdGZvcm0tY29tcHV0ZWQtYXNzZXRzLnMzLXVzLXdlc3QtMi5hbWF6b25hd3MuY

In [0]:
clip_download_url = check_state_request.json()['_links']['results'][0]

In [40]:
print (clip_download_url)

https://api.planet.com/data/v1/download?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE1NTY4NzU0ODMsInN1YiI6InFsdlNuUFpMRGxQTGl6aDZGZUFrOVR2cUN2dXErZGN4SEhhQUlWR3IzbDdWS1huSDR1NmNPRzdPK256aytPSGxwejdjZ1hzUnUxVFBHcnZhWnFDOCtnPT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiJ7XCJ0eXBlXCI6IFwiUG9seWdvblwiLCBcImNvb3JkaW5hdGVzXCI6IFtbWy0xMjIuNTIyMjc3ODMyMDMxMjUsIDQwLjY2MDg0NzY5NzI4NDgxNV0sIFstMTIyLjUyMjI3NzgzMjAzMTI1LCA0MC45ODcxNTQ5MzM3OTczMzVdLCBbLTEyMi4wMTY5MDY3MzgyODEyNCwgNDAuOTg3MTU0OTMzNzk3MzM1XSwgWy0xMjIuMDE2OTA2NzM4MjgxMjQsIDQwLjY2MDg0NzY5NzI4NDgxNV0sIFstMTIyLjUyMjI3NzgzMjAzMTI1LCA0MC42NjA4NDc2OTcyODQ4MTVdXV19IiwiYXNzZXRzIjpbeyJpdGVtX3R5cGUiOiJSRU9ydGhvVGlsZSIsImFzc2V0X3R5cGUiOiJ2aXN1YWwiLCJpdGVtX2lkIjoiMjAxNjA3MDdfMTk1MTQ3XzEwNTc5MTZfUmFwaWRFeWUtMSJ9XSwidXJsIjoiaHR0cHM6Ly9wbGFuZXQtcGxhdGZvcm0tY29tcHV0ZWQtYXNzZXRzLnMzLXVzLXdlc3QtMi5hbWF6b25hd3MuY29tLzc2MGY2NjRjLWU2MjktNDI1Mi1iMGFiLWVhZjc5MjAwZDk1NS1jbGlwcy56aXA_WC1BbXotQWxnb3JpdGhtPUFXUzQtSE1BQy1TSEEyNTZcdTAwMjZYL

In [0]:
import zipfile

In [0]:
from tqdm import tqdm

In [57]:
response = requests.get(clip_download_url, stream=True)
with open(item_id + '.zip', "wb") as handle:
   for data in tqdm(response.iter_content()):
       handle.write(data)

367it [00:00, 51407.99it/s]


In [59]:
zipped_item = zipfile.ZipFile('./'+item_id + '.zip')
zipped_item.extractall(item_id)

BadZipFile: ignored

In [61]:
print(os.listdir())

['.config', '20160707_195147_1057916_RapidEye-1.zip', 'sample_data']


In [0]:
from google.colab import drive
from google.colab import files
files.download("20160707_195147_1057916_RapidEye-1.zip")